In [35]:
import os
import pandas as pd
import openai
from docx import Document
from pptx import Presentation
import time
from transformers import GPT2Tokenizer

In [43]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY #input your key here

In [44]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Functions to help parse text and call GPT

In [239]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def chunk_text(text, max_chunk_size):
    #TODO: improve chunking logic
    words = word_tokenize(text)
    chunks = [words[i:i + max_chunk_size] for i in range(0, len(words), max_chunk_size)]
    return ["".join(chunk) for chunk in chunks]

def chat(input_value, max_tokens, engine = 'text-davinci-003'):
#Note: The openai-python library support for Azure OpenAI is in preview. 
    openai.api_type = "azure"
    openai.api_base = "https://eastus.api.cognitive.microsoft.com/"
    openai.api_version = "2022-12-01"
    openai.api_key = os.getenv("OPENAI_API_KEY")
    
    response = openai.Completion.create(
      engine=engine,
      prompt=f"{input_value}\n\n",
      temperature=1,
      max_tokens=max_tokens,
      top_p=0.5,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
    return response.choices[0].text

def chunk_gpt_response(user_input, analysis_text, max_prompt = 3000, max_response_tokens=1000, engine='text-davinci-003'):
    prompt_max_len = max_prompt - len(word_tokenize(user_input))
    #read in file and convert to text
    chunks = chunk_text(analysis_text, prompt_max_len)
    results = []
    if len(chunks) > 0:
        for chunk in chunks:
            results.append(chat(user_input + chunk, max_response_tokens))
    else:
        results.append(chat(user_input, max_response_tokens))
    return '\n'.join(results)


def word_tokenize(text):
    ids = tokenizer(text)['input_ids']
    return [tokenizer.decode(x) for x in ids]

## Functions for extracting useful information from data

In [240]:
# fits a linear regression model to the data versus the index value
def get_slope(data):
    summaries = {}
    for col in data.columns:
        model = sm.OLS(data[col], sm.tools.add_constant(data.index)).fit()
        summaries[col] = model.summary()
    return summaries

In [241]:
# looks for outliers based on a simple z score
def get_outliers(data):
    summary = data.describe()
    # Calculate Z-score for each data point
    zscore = (data - summary.loc['mean'])/summary.loc['std']
    # Identify data points with Z-score > 3
    outlier_dict = {}
    for col in data.columns:
        outliers = data[col][data[col] > 3]
        outlier_dict[col] = outliers
    return outlier_dict

In [291]:
# for data that appears to be geo-related, do a simple groupby and average to get basic summary values
def geo_analysis(data, data_explanation = ""):
    prompt = 'Given the columns of a dataframe below, return the columns that are related to geography as a python list'
    geo_columns = chat(prompt + str(list(data.columns)), max_tokens = 1000)
    geo_columns = eval(geo_columns)
    results = {}
    for col in geo_columns:
        agg_data = data.groupby(col).mean(numeric_only=True)
        if agg_data.shape[1] == 0:
            continue
        agg_data = agg_data.sort_values(agg_data.columns[0], ascending=False)
        agg_data = agg_data.to_dict()
        results[col] = chunk_gpt_response(data_explanation + "Summarize the key insights from the following data. The data is sorted from highest value to lower.",
                                          str(agg_data), 3000, 1000)
    return results    
    

In [243]:
# sepearate analysis if we're looking at geo vs nongeo data - we would likely want to expand the classification at some point
def is_geo_data(data):
    prompt = "Below is the first few rows of a dataframe. Return 1 if the data is geography-related, and 0 otherwise"
    return chat(prompt + str(data.head()), 10).replace('\n', '')

## Reading in data (i downloaded this manually)

In [244]:
mortality = pd.read_csv("/Users/amitmisra/Downloads/mortality.csv")

In [245]:
# this are examples of views that are showed - not sure if we have the 
mortality_by_country = mortality[mortality.Period == 2019][['ParentLocation', 'Location', 'FactValueNumeric']]
mortality_over_time = mortality.groupby('Period').FactValueNumeric.mean().reset_index()

In [246]:
# for non-geo dataframes, run the slope and outlier analyses as a simple starting point
def non_geo_analysis(data, data_explanation=""):
    slope_results = get_slope(data)
    slope_gpt = {}
    for key in slope_results:
        slope_gpt[key] = chunk_gpt_response(data_explanation + f"Given the following model results for a linear regression model for column {key}, describe the key insights in layment's terms",
                                            str(slope_results[key]), 3000, 1000)
    outlier_results = get_outliers(data)
    outlier_gpt = {}
    for key in outlier_results:
        outlier_gpt[key] = chunk_gpt_response(data_explanation + f"Given the following outliers for column {key}, return the main outliers with their corresponding {key} values:",
                                              str(outlier_results[key]), 1000)
    return slope_gpt, outlier_gpt

In [271]:
# this is the endpoint where you can input a pandas dataframe and an optional explanation of what the data is
# providing an explanation will give better results
def return_insights(data, data_explanation = ""):
    if is_geo_data(data) == "1":
        gpt_results = geo_analysis(data, data_explanation)
    else:
        gpt_results = non_geo_analysis(data, data_explanation)
    summary_prompt = data_explanation + "Identify the key insights from the following analysis of the data and explain in simple terms. The analysis is formatted as a python dictionary with column names as keys. "
    final_result = chunk_gpt_response(summary_prompt, str(gpt_results), 3000, 1000)
    return final_result


In [272]:
print(return_insights(mortality_over_time))


The key insight from this analysis is that the value of FactValueNumeric has decreased significantly over the past 30 years, from 13.37 in 1990 to 5.62 in 2019. This indicates that the value has decreased over the years, with the highest decrease from 1990 to 2019.


In [273]:
print(return_insights(mortality_over_time, "This is data on mortality rates for 5-14 year olds over time. FactValueNumeric is the mortality rate. "))


The key insight from this data is that mortality rates for 5-14 year olds have decreased significantly over time. In 1990, the mortality rate was 13.37 per 100,000 people, but by 2019, it had decreased to 5.62 per 100,000 people. This indicates that mortality rates for this age group have decreased by over 57% in the past 30 years. This suggests that public health initiatives and advances in medical technology have had a positive impact on the mortality rate of this age group.


In [274]:
print(return_insights(mortality_by_country))


The key insights from this analysis are that the highest FactValueNumeric is in Africa, with the highest value being in Niger at 30.29. The lowest FactValueNumeric is in Luxembourg at 0.38. The average FactValueNumeric is 8.82, and the majority of countries have a FactValueNumeric between 10 and 5.


In [275]:
print(return_insights(mortality_by_country, "This is data on mortality rates for 5-14 year olds by country. FactValueNumeric is the mortality rate. "))


The key insights from this analysis are that mortality rates for 5-14 year olds are highest in Africa and lowest in Europe, and that there is a great variation in mortality rates between countries, with Niger having the highest rate and Luxembourg having the lowest.


### Results on raw data without any groupbys

This shows that this methodology works okay to extract information from the raw csv, but only if we give some guidance on what's in the data and what's important. Otherwise the model will return things that are not really significant.

However, once we give some modest direction (which any non-technical person could write), we get results that, while not perfect, do a pretty good job describing the key points from the data)

In [290]:
print(return_insights(mortality))



The key insight from the data is that all countries have the same period of 2004.5 and IsLatestYear of 0.03333333333333333. This indicates that the data is from the same period and is the most recent data available. Additionally, the data is sorted from highest value to lower, providing an overview of the values in descending order. This allows for a comparison of the values between countries and regions.

The key insights from this analysis are: 
1. It provides information about the highest value, range of values, unit of measure, and any comments associated with the facts. 
2. It can be used to analyze trends over time and compare different values and measurements. 
3. It can provide insights into the period, type of dimensions, values, numerical prefixes, numerical values, units of measurement, translations, and comments.


In [289]:
print(return_insights(mortality, "This is data for mortality rates for 5-14 year olds. The key columns are Period (for time), ParentLocation (for country) and FactValueNumeric (the measured mortality rate)"))



The key insights from this data are that mortality rates for 5-14 year olds vary greatly between countries. The highest rate is in Sierra Leone at 25.9 per 1000, while the lowest rate is in Singapore at 0.7 per 1000. On average, the mortality rate for 5-14 year olds is 8.87 per 1000. The data also shows that mortality rates tend to be higher in African and Asian countries, with many countries in these regions having rates above 20 per 1000. The data also suggests that mortality rates are generally higher in developing countries than in developed countries.

The key insights from this data are that mortality rates for 5-14 year olds have been decreasing over time, with the highest mortality rate recorded in 1991 at 13.335589743589743 and the lowest in 2019 at 5.619538461538461. This indicates that there has been a significant improvement in mortality rates over the past two decades. The data also shows that the mortality rate for 5-14 year olds is highest in certain countries, with th